In [1]:
###############################################
# Sandpile Sylow p experiments for bipartite ER graphs
###############################################

from sage.all import *
from collections import Counter
import csv

###############################################
# 1. Sandpile + Sylow p partition
###############################################

def sandpile_invariant_factors(G, sink=None):
    """
    Return the invariant factors [d1, ..., dk] of the sandpile group of G,
    i.e. the non-trivial diagonal entries in the SNF of a reduced Laplacian.
    """
    if sink is None:
        sink = G.vertices()[0]
    S = Sandpile(G, sink)
    inv = S.invariant_factors()
    return [ZZ(d) for d in inv if ZZ(d) != 1]


def p_primary_partition(inv_factors, p):
    """
    Given invariant factors [d1,...,dk] of a finite abelian group,
    return the Sylow p-subgroup as a partition λ of exponents:

        G_p ≅ ⊕ Z/p^{λ_i} Z

    represented as a tuple λ = (λ_1 ≥ λ_2 ≥ ...).
    The empty tuple () corresponds to the trivial p-group.
    """
    p = ZZ(p)
    exponents = []
    for d in inv_factors:
        v = ZZ(d).valuation(p)
        if v > 0:
            exponents.append(v)
    exponents.sort(reverse=True)
    return tuple(exponents)


def sylow_p_partition_of_graph(G, p, sink=None):
    """
    Compute the Sylow p-subgroup of the sandpile group of G,
    and return it as a partition λ.
    """
    inv = sandpile_invariant_factors(G, sink=sink)
    return p_primary_partition(inv, p)


###############################################
# 2. Random bipartite Erdős–Rényi graphs
###############################################

def random_bipartite_ER(n1, alpha, u):
    """
    Erdős–Rényi random bipartite graph G(n1, floor(alpha*n1), u).

    Edges are included independently with probability u between
    the two parts of sizes n1 and n2 = floor(alpha*n1).
    """
    n1 = ZZ(n1)
    alpha = QQ(alpha)
    n2 = floor(alpha * n1)
    n2 = ZZ(n2)
    G = graphs.RandomBipartite(n1, n2, u)
    return G, n2


def sample_bipartite_sylow_p(n1, alpha, u, p, N,
                             require_connected=True,
                             sink=None):
    """
    Sample N graphs G(n1, alpha, u) and record the Sylow p-partition
    of the sandpile group for each.

    Returns a dict:
        {
          'p': p,
          'n1': n1,
          'n2': n2,
          'alpha': alpha,
          'u': u,
          'N': N,
          'samples': N,
          'draws': total_draws,
          'discarded': num_disconnected,
          'counts': Counter mapping λ -> count
        }
    """
    n1 = ZZ(n1)
    alpha = QQ(alpha)
    u = QQ(u)
    counts = Counter()
    draws = 0
    accepted = 0
    discarded = 0

    # Fix n2 from the first graph
    G, n2 = random_bipartite_ER(n1, alpha, u)

    while accepted < N:
        if draws == 0:
            # use the first graph we already generated
            G_cur = G
        else:
            G_cur, n2 = random_bipartite_ER(n1, alpha, u)

        draws += 1

        if require_connected and not G_cur.is_connected():
            discarded += 1
            continue

        lam = sylow_p_partition_of_graph(G_cur, p, sink=sink)
        counts[lam] += 1
        accepted += 1

    return {
        'p': ZZ(p),
        'n1': n1,
        'n2': n2,
        'alpha': alpha,
        'u': u,
        'N': N,
        'samples': accepted,
        'draws': draws,
        'discarded': discarded,
        'counts': counts,
    }


###############################################
# 3. Conjectural distributions on partitions
#    μ^{Sym}_∞ for p>2,  P^{Sym,1}_2 for p=2
###############################################

def n_lambda(lam):
    """
    n(λ) = sum_{j >= 1} (j-1) λ_j.
    lam is a tuple like (3,1,...).
    """
    return sum(i * lam[i] for i in range(len(lam)))  # i=0 corresponds to j=1


def denom_factor(lam, q):
    """
    Compute ∏_i (q^2; q^2)_{ floor(m_i(λ)/2) } where
      (a; r)_k = ∏_{j=0}^{k-1} (1 - a r^j),
    and m_i(λ) is the multiplicity of part i in λ.

    Here we only need the specialisation with a=q^2 and r=q^2.
    """
    mults = Counter(lam)
    R = parent(q)
    prod = R(1)
    for m in mults.values():
        k = m // 2
        for j in range(k):
            prod *= (1 - q**(2 + 2*j))
    return prod


def qpoch_q_q2_infty(q, L=400):
    """
    Approximate (q; q^2)_∞ = ∏_{j>=0} (1 - q^{2j+1})
    by truncating at j<L.
    """
    R = parent(q)
    prod = R(1)
    for j in range(L):
        prod *= (1 - q**(2*j + 1))
    return prod


def qpoch_minus1_q_infty(q, L=400):
    """
    Approximate (-1; q)_∞ = ∏_{j>=0} (1 + q^j)
    by truncating at j<L.
    This is used in the normalising factor for P^{Sym,1}_2.
    """
    R = parent(q)
    prod = R(1)
    for j in range(L):
        prod *= (1 + q**j)
    return prod

def qpoch_minusq_q_infty(q, L=400):
    """
    Approximate (-q; q)_∞ = ∏_{j>=0} (1 + q^{j+1})
    by truncating at j<L.
    """
    R = parent(q)
    prod = R(1)
    for j in range(L):
        prod *= (1 + q**(j+1))
    return prod


# cache for constants and probabilities
_CONJ_CACHE = {}
_CONST_CACHE = {}


def _get_q_and_consts(p, prec=80, L=400):
    """
    For a given prime p, return (q, C_sym, C_norm) where
      - q = 1/p
      - C_sym = (q; q^2)_∞   (always)
      - C_norm = 2           if p=2 (the (-q^{1-k};q)_k with k=1)
               = None        if p>2 (unused)
    """
    p = int(p)
    if p in _CONST_CACHE:
        return _CONST_CACHE[p]

    R = RealField(prec)
    q = R(1) / R(p)
    C_sym = qpoch_q_q2_infty(q, L=L)
    if p == 2:
        C_norm = R(2)   # (-q^{1-1}; q)_1 = (-1; q)_1 = 2
    else:
        C_norm = None

    _CONST_CACHE[p] = (q, C_sym, C_norm)
    return _CONST_CACHE[p]



def conjectural_prob(p, lam, prec=80, L=400):
    """
    Conjectural probability for a partition λ and prime p.

    - If p > 2: μ^{Sym}_∞(λ)
         (the symmetric Cohen–Lenstra measure).
    - If p = 2: P^{Sym,1}_2(λ)
         (the k=1 biased symmetric measure).

    Computed numerically in RealField(prec).
    """
    p = int(p)
    lam = tuple(lam)
    key = (p, lam)

    if key in _CONJ_CACHE:
        return _CONJ_CACHE[key]

    q, C_sym, C_norm = _get_q_and_consts(p, prec=prec, L=L)
    R = parent(q)

    nlam = n_lambda(lam)
    size = sum(lam)
    denom = denom_factor(lam, q)
    mu_sym = C_sym * q**(nlam + size) / denom  # μ^{Sym}_∞

    if p == 2:
        # P^{Sym,1}_2(λ) ∝ 2^{λ'_1} μ^{Sym}_∞(λ), with λ'_1 = l(λ).
        rank = len(lam)             # λ'_1
        prob = (R(p)**rank) * mu_sym / C_norm
    else:
        prob = mu_sym

    _CONJ_CACHE[key] = prob
    return prob

# cache for rank constants
_RANK_CONST_CACHE = {}

def _get_q_and_rank_const(p, prec=80, L=400):
    """
    For rank distribution. Returns (q, C_rank) where:

      - q = 1/p
      - C_rank = (-q; q)_∞ if p>2 (k=0 case)
               = (-1; q)_∞ if p=2 (k=1 case)

    Values are cached.
    """
    p = int(p)
    if p in _RANK_CONST_CACHE:
        return _RANK_CONST_CACHE[p]

    R = RealField(prec)
    q = R(1) / R(p)
    if p == 2:
        C_rank = qpoch_minus1_q_infty(q, L=L)   # (-1; q)_∞
    else:
        C_rank = qpoch_minusq_q_infty(q, L=L)   # (-q; q)_∞
    _RANK_CONST_CACHE[p] = (q, C_rank)
    return _RANK_CONST_CACHE[p]


def rank_prob_conjectural(p, r, prec=80, L=400):
    """
    Conjectural probability that rank = r under the p-Sylow
    limiting measure on partitions:

      - If p is odd:   p_0(r)
      - If p = 2:      p_1(r)

    where (with q = 1/p):
      p_0(r) = 1/(-q;q)_∞ * q^{binom(r+1,2)} / (q;q)_r
      p_1(r) = 1/(-1;q)_∞ * q^{binom(r,2)}   / (q;q)_r
    """
    p = int(p)
    r = int(r)
    q, C_rank = _get_q_and_rank_const(p, prec=prec, L=L)
    R = parent(q)

    # (q;q)_r = ∏_{j=0}^{r-1} (1 - q^{1+j})
    denom = R(1)
    for j in range(r):
        denom *= (1 - q**(1 + j))

    if p == 2:
        exp_val = binomial(r, 2)
    else:
        exp_val = binomial(r + 1, 2)

    num = q**exp_val
    prob = num / (C_rank * denom)
    return prob


def conj_expectation_p_rank(p):
    """
    Conjectural value of E[p^{rank}] under the limiting measure.
    From Conjecture p-Syl_ev:

      - If p is odd: E = 2
      - If p = 2:    E = 3
    """
    p = int(p)
    if p == 2:
        return 3.0
    else:
        return 2.0




###############################################
# 4. Main driver: run experiments + CSV
###############################################

def run_sandpile_experiments(p, n1, N, alpha_u_list,
                             csv_filename,
                             require_connected=True,
                             seed=None,
                             prec=80,
                             L=400):
    """
    Run Sylow-p sandpile experiments for random bipartite ER graphs
    and save results to CSV files.

    Parameters:
        p              – prime
        n1             – size of first part V1
        N              – number of accepted samples per (alpha,u)
        alpha_u_list   – list of (alpha, u) pairs
        csv_filename   – base CSV filename for partition-level data
        require_connected – discard disconnected graphs if True
        seed           – optional random seed
        prec, L        – precision and truncation for q-products

    Outputs (3 files):
        csv_filename                 – partition-level data
        csv_filename with _ranks     – rank distributions
        csv_filename with _expect    – E[p^{rank}] values
    """
    if seed is not None:
        set_random_seed(seed)

    all_results = []

    # --------- run experiments and print summary ----------
    for (alpha, u) in alpha_u_list:
        print("\n====================================")
        print(f"Running experiments for p={p}, alpha={alpha}, u={u}")

        res = sample_bipartite_sylow_p(
            n1=n1,
            alpha=alpha,
            u=u,
            p=p,
            N=N,
            require_connected=require_connected,
            sink=None
        )
        all_results.append(res)

        counts = res['counts']
        total = sum(counts.values())
        print(f"n1={res['n1']}, n2={res['n2']}, samples={res['samples']}, "
              f"draws={res['draws']}, discarded={res['discarded']}")
        print("Partitions λ ordered by decreasing frequency:")

        items = list(counts.items())
        items.sort(key=lambda kv: (-kv[1], kv[0]))  # by decreasing count

        # also compute observed E[p^{rank}] here
        rank_counts = Counter()
        for lam, c in counts.items():
            r = len(lam)
            rank_counts[r] += c
        E_obs = 0.0
        for r, c in rank_counts.items():
            freq_r = float(c) / float(total) if total > 0 else 0.0
            E_obs += (int(p)**r) * freq_r
        E_conj = conj_expectation_p_rank(p)

        for lam, c in items:
            freq = float(c) / float(total) if total > 0 else 0.0
            conj = float(conjectural_prob(p, lam, prec=prec, L=L))
            lam_str = "()" if lam == () else str(lam)
            print(f"  λ = {lam_str:>8}  count = {c:>4}  "
                  f"freq ≈ {freq:.4f}  conjectured ≈ {conj:.4f}")
        print("Observed E[p^rank] ≈ {:.4f}, conjectured ≈ {:.4f}".format(float(E_obs), float(E_conj)))

    # --------- filenames for extra CSVs ----------
    if csv_filename.lower().endswith(".csv"):
        base = csv_filename[:-4]
    else:
        base = csv_filename
    ranks_filename = base + "_ranks.csv"
    expect_filename = base + "_expect.csv"

    # --------- 1) partition-level CSV ----------
    with open(csv_filename, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([
            'p', 'n1', 'n2', 'N', 'alpha', 'u',
            'partition', 'count', 'frequency', 'conjectured_prob'
        ])

        for res in all_results:
            counts = res['counts']
            total = sum(counts.values())
            p_val = int(res['p'])
            n1_val = int(res['n1'])
            n2_val = int(res['n2'])
            N_val = int(res['samples'])

            alpha_float = float(QQ(res['alpha']))
            u_float = float(QQ(res['u']))
            alpha_str = f"{alpha_float:.6f}"
            u_str = f"{u_float:.6f}"

            for lam, c in counts.items():
                freq = float(c) / float(total) if total > 0 else 0.0
                conj = float(conjectural_prob(p_val, lam, prec=prec, L=L))
                if lam == ():
                    part_str = "[]"
                else:
                    part_str = "[" + ",".join(str(x) for x in lam) + "]"

                writer.writerow([
                    p_val, n1_val, n2_val, N_val,
                    alpha_str, u_str,
                    part_str, int(c), freq, conj
                ])

    print(f"\nPartition-level CSV written to: {csv_filename}")

    # --------- 2) rank distribution CSV ----------
    with open(ranks_filename, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([
            'p', 'n1', 'n2', 'N', 'alpha', 'u',
            'rank', 'count', 'frequency', 'conjectured_prob_rank'
        ])

        for res in all_results:
            counts = res['counts']
            total = sum(counts.values())
            p_val = int(res['p'])
            n1_val = int(res['n1'])
            n2_val = int(res['n2'])
            N_val = int(res['samples'])

            alpha_float = float(QQ(res['alpha']))
            u_float = float(QQ(res['u']))
            alpha_str = f"{alpha_float:.6f}"
            u_str = f"{u_float:.6f}"

            rank_counts = Counter()
            for lam, c in counts.items():
                r = len(lam)
                rank_counts[r] += c

            # we could sort ranks
            for r in sorted(rank_counts.keys()):
                c = rank_counts[r]
                freq_r = float(c) / float(total) if total > 0 else 0.0
                conj_r = float(rank_prob_conjectural(p_val, r, prec=prec, L=L))
                writer.writerow([
                    p_val, n1_val, n2_val, N_val,
                    alpha_str, u_str,
                    int(r), int(c), freq_r, conj_r
                ])

    print(f"Rank distribution CSV written to: {ranks_filename}")

    # --------- 3) expectation CSV ----------
    with open(expect_filename, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([
            'p', 'n1', 'n2', 'N', 'alpha', 'u',
            'observed_E_p_rank', 'conjectured_E_p_rank'
        ])

        for res in all_results:
            counts = res['counts']
            total = sum(counts.values())
            p_val = int(res['p'])
            n1_val = int(res['n1'])
            n2_val = int(res['n2'])
            N_val = int(res['samples'])

            alpha_float = float(QQ(res['alpha']))
            u_float = float(QQ(res['u']))
            alpha_str = f"{alpha_float:.6f}"
            u_str = f"{u_float:.6f}"

            rank_counts = Counter()
            for lam, c in counts.items():
                r = len(lam)
                rank_counts[r] += c

            E_obs = 0.0
            for r, c in rank_counts.items():
                freq_r = float(c) / float(total) if total > 0 else 0.0
                E_obs += (p_val**r) * freq_r

            E_conj = conj_expectation_p_rank(p_val)

            writer.writerow([
                p_val, n1_val, n2_val, N_val,
                alpha_str, u_str,
                E_obs, E_conj
            ])

    print(f"Expectation CSV written to: {expect_filename}")



In [2]:
p = 2
n1 = 100
N = 500
alpha_u_list = [
    (0.45, 0.5),
    (0.60, 0.5),
    (0.80, 0.5),
    (0.70, 0.2),
    (0.70, 0.8),
]
csv_filename = "sandpile_results_p2.csv"

run_sandpile_experiments(p, n1, N, alpha_u_list, csv_filename,
                         require_connected=True, seed=2025)



Running experiments for p=2, alpha=0.450000000000000, u=0.500000000000000
n1=100, n2=45, samples=500, draws=500, discarded=0
Partitions λ ordered by decreasing frequency:
  λ =     (1,)  count =   29  freq ≈ 0.0580  conjectured ≈ 0.2097
  λ =   (1, 1)  count =   24  freq ≈ 0.0480  conjectured ≈ 0.1398
  λ = (1, 1, 1, 1, 1, 1)  count =   23  freq ≈ 0.0460  conjectured ≈ 0.0000
  λ = (1, 1, 1, 1)  count =   22  freq ≈ 0.0440  conjectured ≈ 0.0047
  λ = (1, 1, 1, 1, 1, 1, 1)  count =   19  freq ≈ 0.0380  conjectured ≈ 0.0000
  λ = (1, 1, 1)  count =   17  freq ≈ 0.0340  conjectured ≈ 0.0350
  λ = (1, 1, 1, 1, 1)  count =   16  freq ≈ 0.0320  conjectured ≈ 0.0003
  λ = (1, 1, 1, 1, 1, 1, 1, 1, 1, 1)  count =   13  freq ≈ 0.0260  conjectured ≈ 0.0000
  λ =       ()  count =   11  freq ≈ 0.0220  conjectured ≈ 0.2097
  λ = (1, 1, 1, 1, 1, 1, 1, 1)  count =   11  freq ≈ 0.0220  conjectured ≈ 0.0000
  λ = (1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)  count =   11  freq ≈ 0.0220  conjectured ≈ 0.0000
 

n1=100, n2=60, samples=500, draws=500, discarded=0
Partitions λ ordered by decreasing frequency:
  λ =       ()  count =  108  freq ≈ 0.2160  conjectured ≈ 0.2097
  λ =     (1,)  count =  102  freq ≈ 0.2040  conjectured ≈ 0.2097
  λ =   (1, 1)  count =   72  freq ≈ 0.1440  conjectured ≈ 0.1398
  λ =     (2,)  count =   52  freq ≈ 0.1040  conjectured ≈ 0.1049
  λ =     (3,)  count =   37  freq ≈ 0.0740  conjectured ≈ 0.0524
  λ =   (2, 1)  count =   29  freq ≈ 0.0580  conjectured ≈ 0.0524
  λ =     (4,)  count =   19  freq ≈ 0.0380  conjectured ≈ 0.0262
  λ = (1, 1, 1)  count =   15  freq ≈ 0.0300  conjectured ≈ 0.0350
  λ =   (3, 1)  count =    9  freq ≈ 0.0180  conjectured ≈ 0.0262
  λ = (2, 1, 1)  count =    7  freq ≈ 0.0140  conjectured ≈ 0.0175
  λ =   (4, 1)  count =    7  freq ≈ 0.0140  conjectured ≈ 0.0131
  λ =   (2, 2)  count =    5  freq ≈ 0.0100  conjectured ≈ 0.0175
  λ =   (3, 2)  count =    4  freq ≈ 0.0080  conjectured ≈ 0.0066
  λ =     (5,)  count =    4  freq ≈ 0.0080

In [3]:
p = 3
n1 = 100
N = 500
alpha_u_list = [
    (0.3, 0.5),
    (0.4, 0.5),
    (0.7, 0.5),
    (0.7, 0.2),
    (0.7, 0.8),
]
csv_filename = "sandpile_results_p3.csv"

run_sandpile_experiments(p, n1, N, alpha_u_list, csv_filename,
                         require_connected=True, seed=2025)



Running experiments for p=3, alpha=0.300000000000000, u=0.500000000000000
n1=100, n2=30, samples=500, draws=500, discarded=0
Partitions λ ordered by decreasing frequency:
  λ =       ()  count =   58  freq ≈ 0.1160  conjectured ≈ 0.6390
  λ =     (1,)  count =   39  freq ≈ 0.0780  conjectured ≈ 0.2130
  λ = (1, 1, 1)  count =   38  freq ≈ 0.0760  conjectured ≈ 0.0010
  λ =   (1, 1)  count =   30  freq ≈ 0.0600  conjectured ≈ 0.0266
  λ = (1, 1, 1, 1, 1, 1, 1)  count =   29  freq ≈ 0.0580  conjectured ≈ 0.0000
  λ = (1, 1, 1, 1)  count =   26  freq ≈ 0.0520  conjectured ≈ 0.0000
  λ = (1, 1, 1, 1, 1)  count =   26  freq ≈ 0.0520  conjectured ≈ 0.0000
  λ = (1, 1, 1, 1, 1, 1, 1, 1)  count =   24  freq ≈ 0.0480  conjectured ≈ 0.0000
  λ = (1, 1, 1, 1, 1, 1)  count =   21  freq ≈ 0.0420  conjectured ≈ 0.0000
  λ =     (2,)  count =   17  freq ≈ 0.0340  conjectured ≈ 0.0710
  λ = (1, 1, 1, 1, 1, 1, 1, 1, 1)  count =   16  freq ≈ 0.0320  conjectured ≈ 0.0000
  λ = (2, 1, 1, 1, 1, 1, 1)  cou

n1=100, n2=70, samples=500, draws=500, discarded=0
Partitions λ ordered by decreasing frequency:
  λ =       ()  count =  310  freq ≈ 0.6200  conjectured ≈ 0.6390
  λ =     (1,)  count =  109  freq ≈ 0.2180  conjectured ≈ 0.2130
  λ =     (2,)  count =   36  freq ≈ 0.0720  conjectured ≈ 0.0710
  λ =   (1, 1)  count =   14  freq ≈ 0.0280  conjectured ≈ 0.0266
  λ =     (3,)  count =   14  freq ≈ 0.0280  conjectured ≈ 0.0237
  λ =     (4,)  count =   10  freq ≈ 0.0200  conjectured ≈ 0.0079
  λ =   (3, 1)  count =    3  freq ≈ 0.0060  conjectured ≈ 0.0026
  λ =   (2, 1)  count =    2  freq ≈ 0.0040  conjectured ≈ 0.0079
  λ =   (2, 2)  count =    1  freq ≈ 0.0020  conjectured ≈ 0.0010
  λ =   (5, 1)  count =    1  freq ≈ 0.0020  conjectured ≈ 0.0003
Observed E[p^rank] ≈ 2.0120, conjectured ≈ 2.0000

Running experiments for p=3, alpha=0.700000000000000, u=0.800000000000000
n1=100, n2=70, samples=500, draws=500, discarded=0
Partitions λ ordered by decreasing frequency:
  λ =       ()  count